In [ ]:
from huggingface_hub import InferenceClient
import PIL
import numpy as np

In [ ]:
import pandas as pd
df=pd.read_csv(r"/content/influencers_data.csv")


<ipython-input-14-ac7e86b77d79>:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r"/content/influencers_data.csv")


In [ ]:
df=df[['content']]

In [ ]:
df.isnull().sum()
df.dropna(inplace=True)

In [ ]:
def chk(text):
  if len(text)<120:
    text=np.nan
  return text
df=df.apply(lambda x: x.apply(chk))

In [ ]:
df.isnull().sum()
df.dropna(inplace=True)

In [ ]:
df.shape

(23400, 1)

In [ ]:
next_900 = df.iloc[2710:3650]

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key="YOUR_HF_API")
def change(post):
  post=str(post)
  messages = [
      {"role": "system", "content": "Generate a single prompt that can be used to generate the following text post. Just return the prompt no leading or trailing messages"},
      {"role": "user", "content": post},
  ]

  completion = client.chat.completions.create(
      model="meta-llama/Llama-3.2-3B-Instruct",
    messages=messages,
    max_tokens=500
  )

  prompt=completion.choices[0].message.content
  return prompt

In [ ]:
next_900['prompt']=next_900['content'].apply(lambda x: change(x))

<ipython-input-47-97edabee6df7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  next_900['prompt']=next_900['content'].apply(lambda x: change(x))


In [ ]:
next_900.head()

,content,prompt
3351,A great summary of what hyperfocus means and h...,What are some strategies and techniques you ca...
3352,I've been in touch with the A.I. group of the ...,Describe your positive experience with a recen...
3353,See how many sleep trackers I have tested over...,What would be the unintended consequences of r...
3354,A variety of forces are shaping the healthcare...,What role will emerging technologies and evolv...
3355,Digital Health in China! How could a country k...,How can China leverage its technological advan...


In [ ]:
import pandas as pd
from langdetect import detect, LangDetectException

# Function to detect if a text is in English
def is_english(text):
    try:
        # Detect the language
        lang = detect(text)
        # Return True if language is English, False otherwise
        return lang == "en"
    except LangDetectException:
        # If language detection fails (empty or corrupt text), assume non-English
        return False

# Filter rows where the text in the 'content' column is in English
df_filtered = next_900[next_900['content'].apply(is_english)]

# Save the filtered result to a new CSV file
df_filtered.to_csv("filtered_english4.csv", index=False)

print("Filtered CSV file saved as 'filtered_english2.csv'")


Filtered CSV file saved as 'filtered_english2.csv'
